<a href="https://colab.research.google.com/github/BayramAlpSahin/EE491-492Project/blob/main/16BitBinaryAutoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
from torch import nn

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.GELU(),
            nn.Linear(32, 10),
            nn.GELU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 128),
            nn.GELU(),
            nn.Linear(128, 64),
            nn.GELU(),
            nn.Linear(64, 16),
            nn.Sigmoid(),
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model = Autoencoder().to("cuda")
print(model)

In [ ]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
batch_size = 1000000
epochs = 100000
def train_loop(model, loss_fn, optimizer, t):
    model.train()
    X = torch.randint(0, 2, (batch_size, 16))
    X = X.float()
    X = X.to("cuda")
    pred = model(X)
    loss = loss_fn(pred, X)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if(t % 100 == 0):
        print(f"Epoch {t/100+1}\n-------------------------------")
        print(f"Loss: {loss.item()}")


In [ ]:
for t in range(epochs):
    train_loop(model, loss_fn, optimizer, t)
print("Done!")

In [ ]:
X = torch.randint(0, 2, (100000, 16))
X = X.float()
X = X.to("cuda")
pred = model(X)
pred = torch.round(pred)
num_err = [0]*16
for i in range(100000):
    falses = 0
    for j in range(16):
        if(pred[i][j] != X[i][j]):
            falses += 1
    num_err[falses] += 1
for i in range(16):
  num_err[i] /= 100000
print(num_err)
print(X[100])
print(pred[100])


In [ ]:
torch.save(model, "model3.pth")